<a href="https://colab.research.google.com/github/KHUSHIHN/python/blob/master/Week_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import time
import itertools
import heapq
from collections import deque, defaultdict

GOAL = (1, 2, 3, 4, 5, 6, 7, 8, 0)
MOVES = {
    0: (1, 3),
    1: (0, 2, 4),
    2: (1, 5),
    3: (0, 4, 6),
    4: (1, 3, 5, 7),
    5: (2, 4, 8),
    6: (3, 7),
    7: (4, 6, 8),
    8: (5, 7)
}

def is_solvable(state):
    arr = [x for x in state if x != 0]
    inv = sum(1 for i in range(len(arr)) for j in range(i + 1, len(arr)) if arr[i] > arr[j])
    return inv % 2 == 0

def neighbors(state):
    z = state.index(0)
    for nb in MOVES[z]:
        s = list(state)
        s[z], s[nb] = s[nb], s[z]
        yield tuple(s), 1  # step cost 1

def random_start(max_shuffles=60):
    s = list(GOAL)
    z = s.index(0)
    for _ in range(random.randint(20, max_shuffles)):
        choices = MOVES[z]
        nb = random.choice(choices)
        s[z], s[nb] = s[nb], s[z]
        z = nb
    return tuple(s)

# ---------- Generic search framework ----------

class Result:
    def __init__(self, found, path_cost, nodes_expanded, depth, secs, path):
        self.found = found
        self.path_cost = path_cost
        self.nodes_expanded = nodes_expanded
        self.depth = depth
        self.secs = secs
        self.path = path

def reconstruct(parents, s):
    path = [s]
    while s in parents:
        s = parents[s]
        path.append(s)
    path.reverse()
    return path

def run_search(start, algo="bfs", heuristic=None):
    t0 = time.perf_counter()
    nodes_expanded = 0

    if start == GOAL:
        return Result(True, 0, 0, 0, 0.0, [start])

    if algo == "bfs":
        Q = deque([start])
        parents = {}
        seen = {start}

        while Q:
            s = Q.popleft()
            nodes_expanded += 1
            for nxt, c in neighbors(s):
                if nxt in seen:
                    continue
                parents[nxt] = s
                seen.add(nxt)
                if nxt == GOAL:
                    path = reconstruct(parents, nxt)
                    return Result(True, len(path) - 1, nodes_expanded, len(path) - 1,
                                  time.perf_counter() - t0, path)
                Q.append(nxt)

        return Result(False, -1, nodes_expanded, -1, time.perf_counter() - t0, [])

    if algo == "dfs":
        stack = [start]
        parents = {}
        seen = {start}

        while stack:
            s = stack.pop()
            nodes_expanded += 1
            for nxt, c in neighbors(s):
                if nxt in seen:
                    continue
                parents[nxt] = s
                seen.add(nxt)
                if nxt == GOAL:
                    path = reconstruct(parents, nxt)
                    return Result(True, len(path) - 1, nodes_expanded, len(path) - 1,
                                  time.perf_counter() - t0, path)
                stack.append(nxt)

        return Result(False, -1, nodes_expanded, -1, time.perf_counter() - t0, [])

    if algo in ("ucs", "astar"):
        g = defaultdict(lambda: float("inf"))
        g[start] = 0
        parents = {}
        cnt = itertools.count()

        def h(s):
            if heuristic is None:
                return 0
            return heuristic(s)

        open_heap = [(g[start] + h(start), next(cnt), start)]
        closed = set()

        while open_heap:
            f, _, s = heapq.heappop(open_heap)
            if s in closed:
                continue
            closed.add(s)
            nodes_expanded += 1

            if s == GOAL:
                path = reconstruct(parents, s)
                return Result(True, g[s], nodes_expanded, len(path) - 1,
                              time.perf_counter() - t0, path)

            for nxt, c in neighbors(s):
                tentative = g[s] + c
                if tentative < g[nxt]:
                    g[nxt] = tentative
                    parents[nxt] = s
                    heapq.heappush(open_heap,
                                   (tentative + (0 if algo == "ucs" else h(nxt)),
                                    next(cnt), nxt))

        return Result(False, -1, nodes_expanded, -1, time.perf_counter() - t0, [])

    raise ValueError("Unknown algo")


In [ ]:
import random
import time
import heapq
import itertools
from collections import deque, defaultdict
import statistics as st

# ---------- Puzzle model ----------
GOAL = (1, 2, 3, 4, 5, 6, 7, 8, 0)
MOVES = {
    0: (1, 3),
    1: (0, 2, 4),
    2: (1, 5),
    3: (0, 4, 6),
    4: (1, 3, 5, 7),
    5: (2, 4, 8),
    6: (3, 7),
    7: (4, 6, 8),
    8: (5, 7)
}

def is_solvable(state):
    arr = [x for x in state if x != 0]
    inv = sum(1 for i in range(len(arr)) for j in range(i + 1, len(arr)) if arr[i] > arr[j])
    return inv % 2 == 0

def neighbors(state):
    z = state.index(0)
    for nb in MOVES[z]:
        s = list(state)
        s[z], s[nb] = s[nb], s[z]
        yield tuple(s), 1  # step cost 1

def random_start(max_shuffles=60):
    s = list(GOAL)
    z = s.index(0)
    for _ in range(random.randint(20, max_shuffles)):
        choices = MOVES[z]
        nb = random.choice(choices)
        s[z], s[nb] = s[nb], s[z]
        z = nb
    return tuple(s)

# ---------- Generic search framework ----------
class Result:
    def __init__(self, found, path_cost, nodes_expanded, depth, secs, path):
        self.found = found
        self.path_cost = path_cost
        self.nodes_expanded = nodes_expanded
        self.depth = depth
        self.secs = secs
        self.path = path

def reconstruct(parents, s):
    path = [s]
    while s in parents:
        s = parents[s]
        path.append(s)
    path.reverse()
    return path

def run_search(start, algo="bfs", heuristic=None):
    t0 = time.perf_counter()
    nodes_expanded = 0

    if start == GOAL:
        return Result(True, 0, 0, 0, 0.0, [start])

    if algo == "bfs":
        Q = deque([start])
        parents = {}
        seen = {start}
        while Q:
            s = Q.popleft()
            nodes_expanded += 1
            for nxt, c in neighbors(s):
                if nxt in seen:
                    continue
                parents[nxt] = s
                seen.add(nxt)
                if nxt == GOAL:
                    path = reconstruct(parents, nxt)
                    return Result(True, len(path) - 1, nodes_expanded, len(path) - 1, time.perf_counter() - t0, path)
                Q.append(nxt)
        return Result(False, -1, nodes_expanded, -1, time.perf_counter() - t0, [])

    if algo == "dfs":
        stack = [start]
        parents = {}
        seen = {start}
        while stack:
            s = stack.pop()
            nodes_expanded += 1
            for nxt, c in neighbors(s):
                if nxt in seen:
                    continue
                parents[nxt] = s
                seen.add(nxt)
                if nxt == GOAL:
                    path = reconstruct(parents, nxt)
                    return Result(True, len(path) - 1, nodes_expanded, len(path) - 1, time.perf_counter() - t0, path)
                stack.append(nxt)
        return Result(False, -1, nodes_expanded, -1, time.perf_counter() - t0, [])

    if algo in ("ucs", "astar"):
        g = defaultdict(lambda: float("inf"))
        g[start] = 0
        parents = {}
        cnt = itertools.count()

        def h(s):
            return 0 if heuristic is None else heuristic(s)

        open_heap = [(g[start] + h(start), next(cnt), start)]
        closed = set()
        while open_heap:
            f, _, s = heapq.heappop(open_heap)
            if s in closed:
                continue
            closed.add(s)
            nodes_expanded += 1
            if s == GOAL:
                path = reconstruct(parents, s)
                return Result(True, g[s], nodes_expanded, len(path) - 1, time.perf_counter() - t0, path)

            for nxt, c in neighbors(s):
                tentative = g[s] + c
                if tentative < g[nxt]:
                    g[nxt] = tentative
                    parents[nxt] = s
                    heapq.heappush(open_heap, (tentative + (0 if algo == "ucs" else h(nxt)), next(cnt), nxt))

        return Result(False, -1, nodes_expanded, -1, time.perf_counter() - t0, [])

    raise ValueError("Unknown algo")

# ---------- Heuristics ----------
goal_pos = {v: i for i, v in enumerate(GOAL)}

def manhattan(s):
    dist = 0
    for idx, val in enumerate(s):
        if val == 0:
            continue
        gi = goal_pos[val]
        r1, c1 = divmod(idx, 3)
        r2, c2 = divmod(gi, 3)
        dist += abs(r1 - r2) + abs(c1 - c2)
    return dist

def linear_conflict(s):
    dist = manhattan(s)
    # Rows
    for r in range(3):
        row = s[3*r:3*r+3]
        goal_row = [1+3*r, 2+3*r, 3+3*r]
        tiles = [x for x in row if x in goal_row]
        for i in range(len(tiles)):
            for j in range(i+1, len(tiles)):
                if goal_row.index(tiles[i]) > goal_row.index(tiles[j]):
                    dist += 2
    # Columns
    for c in range(3):
        col = s[c::3]
        goal_col = [c+1, c+4, c+7]
        tiles = [x for x in col if x in goal_col]
        for i in range(len(tiles)):
            for j in range(i+1, len(tiles)):
                if goal_col.index(tiles[i]) > goal_col.index(tiles[j]):
                    dist += 2
    return dist

# ---------- Experiments ----------
def run_experiments(num=5, seed=7):
    random.seed(seed)
    starts = []
    while len(starts) < num:
        s = random_start()
        if is_solvable(s) and s not in starts:
            starts.append(s)

    rows = []
    for i, s in enumerate(starts, 1):
        for algo in ["bfs", "dfs", "ucs"]:
            res = run_search(s, algo=algo)
            rows.append(("Case"+str(i), algo.upper(), res.path_cost, res.nodes_expanded, res.secs))
        for name, h in [("A*-Manhattan", manhattan), ("A*-LinearConflict", linear_conflict)]:
            res = run_search(s, algo="astar", heuristic=h)
            rows.append(("Case"+str(i), name, res.path_cost, res.nodes_expanded, res.secs))
    return rows

if __name__ == "__main__":
    rows = run_experiments()
    print("Case, Method, PathCost, NodesExpanded, Time(s)")
    for r in rows:
        print(",".join([str(x) for x in r]))

    # Aggregate UCS vs A*
    def agg(label):
        subset = [r for r in rows if r[1] == label]
        return st.mean([r[3] for r in subset]), st.mean([r[4] for r in subset])

    u_nodes, u_time = agg("UCS")
    m_nodes, m_time = agg("A*-Manhattan")
    l_nodes, l_time = agg("A*-LinearConflict")
    print("\nAverages over 5 starts:")
    print(f"UCS : nodes={u_nodes:.0f}, time={u_time:.4f}s")
    print(f"A* M : nodes={m_nodes:.0f}, time={m_time:.4f}s")
    print(f"A* LC: nodes={l_nodes:.0f}, time={l_time:.4f}s")

    # Heuristic notes
    print("\nHeuristic notes:")
    print("- Manhattan is admissible & consistent for 8-puzzle.")
    print("- Linear-Conflict = Manhattan + 2 per pair in row/col with reversed goal order; still admissible & consistent.")


Case, Method, PathCost, NodesExpanded, Time(s)
Case1,BFS,10,371,0.0007439139999405597
Case1,DFS,20688,23224,0.04804700900012904
Case1,UCS,10,601,0.0018816699998751574
Case1,A*-Manhattan,10,26,0.0002011430001402914
Case1,A*-LinearConflict,10,23,0.00038219399993977277
Case2,BFS,4,13,2.9214000278443564e-05
Case2,DFS,214,232,0.0004184590002296318
Case2,UCS,4,27,7.677499979763525e-05
Case2,A*-Manhattan,4,5,3.7786000120831886e-05
Case2,A*-LinearConflict,4,5,0.00010206900014964049
Case3,BFS,20,34664,0.06243827400021473
Case3,DFS,22492,25224,0.05272412400017856
Case3,UCS,20,50118,0.20720023799958653
Case3,A*-Manhattan,20,756,0.004829612999856181
Case3,A*-LinearConflict,20,437,0.005917324000165536
Case4,BFS,17,9418,0.015886564000084036
Case4,DFS,47999,54877,0.127306655999746
Case4,UCS,17,14346,0.07821021800009476
Case4,A*-Manhattan,17,80,0.0005429039997579821
Case4,A*-LinearConflict,17,63,0.0009076149999600602
Case5,BFS,4,14,3.0387999686354306e-05
Case5,DFS,46136,52626,0.1137659339997299
Case5,

In [ ]:
import heapq

def a_star_search(initial_board, goal_board):
    initial_state = State(initial_board)
    frontier = [(heuristic(initial_state, goal_board), initial_state)]  # (f_cost, state)
    explored = set()

    while frontier:
        f_cost, current_state = heapq.heappop(frontier)

        if is_goal(current_state, goal_board):
            # Reconstruct path and return
            path = []
            while current_state:
                path.append(current_state.action)
                current_state = current_state.parent
            return path[::-1][1:] # Exclude None from initial state

        explored.add(current_state)

        for successor in get_successors(current_state):
            if successor not in explored:
                heapq.heappush(frontier, (successor.cost + heuristic(successor, goal_board), successor))

    return None # No solution found

In [ ]:
from collections import deque
from itertools import product

# 4x4 coordinates: (1..4, 1..4)
DIRS = [(1,0), (-1,0), (0,1), (0,-1)]

def in_bounds(x,y):
    return 1 <= x <= 4 and 1 <= y <= 4

def neighbors(x,y):
    for dx,dy in DIRS:
        nx,ny = x+dx, y+dy
        if in_bounds(nx,ny):
            yield (nx,ny)

# ---------------- KB ----------------
class KB:
    """
    Lightweight propositional KB for standard pit/breeze rules:
    - ¬B(x,y) ⇒ all neighbors are ¬Pit
    - B(x,y) ⇒ at least one neighbor is Pit (we keep {possible_pit} set)
    - If a neighbor becomes known-safe, remove from possible pits
    - If only one 'possible pit' remains around a breezy cell ⇒ that cell is Pit
    """
    def __init__(self):
        self.safe = set()     # proven safe cells
        self.pit = set()      # proven pits
        self.possible = {}    # (x,y) with breeze -> set of candidate pit cells
        self.logs = []

    def assert_no_breeze(self, x, y):
        # infer all neighbors are safe
        for n in neighbors(x, y):
            if n not in self.safe and n not in self.pit:
                self.safe.add(n)
                self.logs.append(f"From ¬B({x},{y}) infer SAFE{n}")

    def assert_breeze(self, x, y):
        if (x, y) not in self.possible:
            cands = {n for n in neighbors(x,y) if n not in self.safe and n not in self.pit}
            self.possible[(x, y)] = cands
            self.logs.append(f"B({x},{y}) ⇒ at least one of {sorted(cands)} is a Pit")

    def mark_safe(self, cell):
        if cell in self.safe:
            return
        self.safe.add(cell)
        # remove from all candidate sets
        for k in list(self.possible):
            if cell in self.possible[k]:
                self.possible[k].remove(cell)
                self.logs.append(f"Since {cell} is SAFE, remove from pit-candidates of {k}")

    def deduce(self):
        changed = True
        while changed:
            changed = False
            # If any breezy cell has all but one neighbor eliminated ⇒ that remaining neighbor is a Pit
            for k, cands in list(self.possible.items()):
                cands = {c for c in cands if c not in self.safe and c not in self.pit}
                self.possible[k] = cands
                if len(cands) == 1:
                    p = next(iter(cands))
                    if p not in self.pit:
                        self.pit.add(p)
                        changed = True
                        self.logs.append(f"Only {p} fits B{tuple(k)} ⇒ PIT{p}")
                elif len(cands) == 0:
                    # contradiction handled softly; ignore
                    pass

# ---------------- World ----------------
class World:
    """
    Simple world with:
    - exactly one Wumpus
    - some pits
    - one gold cell
    Agent perceives Breeze (near pits) and Stench (near Wumpus).
    """
    def __init__(self, pits={(3,1)}, wumpus=(4,3), gold=(2,3)):
        self.pits = set(pits)
        self.wumpus = wumpus
        self.gold = gold

    def percept(self, x, y):
        breeze = any(n in self.pits for n in neighbors(x,y))
        stench = any(n == self.wumpus for n in neighbors(x,y))
        glitter = (x,y) == self.gold
        return {"breeze": breeze, "stench": stench, "glitter": glitter}

# ---------------- Agent ----------------
def agent_run(world, start=(1,1)):
    kb = KB()
    path = [start]
    visited = set([start])
    have_gold = False

    def process_cell(c):
        nonlocal have_gold
        x,y = c
        p = world.percept(x,y)
        if p["glitter"]:
            have_gold = True
            kb.logs.append(f"GLITTER at {c} ⇒ grab gold")

        if p["breeze"]:
            kb.assert_breeze(x,y)
        else:
            kb.assert_no_breeze(x,y)
        kb.mark_safe(c)
        kb.deduce()

    # Process start
    process_cell(start)

    # Explore: move to any neighbor known safe & unvisited
    while True:
        options = [n for n in neighbors(*path[-1]) if n in kb.safe and n not in visited]
        if not options:
            break
        nxt = sorted(options)[0]
        path.append(nxt)
        visited.add(nxt)
        process_cell(nxt)

        # If gold grabbed, head back to (1,1)
        if have_gold and nxt != (1,1):
            def manhattan(a,b): return abs(a[0]-b[0])+abs(a[1]-b[1])
            current = nxt
            while current != (1,1):
                cands = [n for n in neighbors(*current) if n in kb.safe]
                if not cands: break
                best = min(cands, key=lambda t: (manhattan(t,(1,1)), t))
                path.append(best)
                visited.add(best)
                current = best
            break

    return {
        "path": path,
        "have_gold": have_gold,
        "safe": sorted(kb.safe),
        "pits_inferred": sorted(kb.pit),
        "log": kb.logs
    }

# ---------------- Run example ----------------
if __name__ == "__main__":
    world = World(pits={(3,1),(3,3)}, wumpus=(4,3), gold=(2,3))
    out = agent_run(world)
    print("PATH:", out["path"])
    print("GOT GOLD:", out["have_gold"])
    print("SAFE:", out["safe"])
    print("INFERRED PITS:", out["pits_inferred"])
    print("\nDerived sentences:")
    for line in out["log"]:
        print("-", line)


PATH: [(1, 1), (1, 2), (1, 3), (1, 4), (2, 4), (2, 3), (1, 3), (1, 2), (1, 1)]
GOT GOLD: True
SAFE: [(1, 1), (1, 2), (1, 3), (1, 4), (2, 1), (2, 2), (2, 3), (2, 4), (3, 4)]
INFERRED PITS: [(3, 3)]

Derived sentences:
- From ¬B(1,1) infer SAFE(2, 1)
- From ¬B(1,1) infer SAFE(1, 2)
- From ¬B(1,2) infer SAFE(2, 2)
- From ¬B(1,2) infer SAFE(1, 3)
- From ¬B(1,3) infer SAFE(2, 3)
- From ¬B(1,3) infer SAFE(1, 4)
- From ¬B(1,4) infer SAFE(2, 4)
- From ¬B(2,4) infer SAFE(3, 4)
- GLITTER at (2, 3) ⇒ grab gold
- B(2,3) ⇒ at least one of [(3, 3)] is a Pit
- Only (3, 3) fits B(2, 3) ⇒ PIT(3, 3)


In [ ]:
import json, os, sys, numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.utils import Bunch

# ---------------- Dataset Loader ----------------
def load_dataset():
    """
    Try California Housing; if fetch fails (e.g., no internet), fallback to Diabetes
    """
    try:
        from sklearn.datasets import fetch_california_housing
        ds = fetch_california_housing(as_frame=True)
        name = "California Housing"
        X = ds.frame.drop(columns=[ds.target_names[0]])
        y = ds.frame[ds.target_names[0]]
        feature_names = list(X.columns)
        return Bunch(data=X, target=y, feature_names=feature_names, name=name)
    except Exception:
        from sklearn.datasets import load_diabetes
        ds = load_diabetes(as_frame=True)
        name = "Diabetes (fallback)"
        X = ds.frame.drop(columns=["target"])
        y = ds.frame["target"]
        feature_names = list(X.columns)
        return Bunch(data=X, target=y, feature_names=feature_names, name=name)

# ---------------- Quick EDA ----------------
def quick_eda(X, y, name):
    print(f"Dataset: {name} | samples={len(X)} | features={len(X.columns)}")
    print("Head:\n", X.head(3))
    print("Target stats: mean={:.3f} std={:.3f}".format(y.mean(), y.std()))

    # Tiny histogram
    plt.figure()
    y.hist(bins=30)
    plt.title(f"{name} target distribution")
    plt.xlabel("y")
    plt.ylabel("count")
    plt.tight_layout()
    plt.savefig("target_hist.png")
    plt.close()

# ---------------- Build and Evaluate ----------------
def build_and_eval(X, y, feature_names):
    num_features = list(feature_names)

    pre = ColumnTransformer(
        transformers=[("num", StandardScaler(with_mean=True, with_std=True), num_features)],
        remainder="drop"
    )

    pipe = Pipeline([("pre", pre), ("lr", LinearRegression())])

    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)
    pipe.fit(Xtr, ytr)

    preds = pipe.predict(Xte)

    rmse = np.sqrt(mean_squared_error(yte, preds))
    mae = mean_absolute_error(yte, preds)

    print(f"RMSE: {rmse:.4f} | MAE: {mae:.4f}")

    # Pred vs True plot
    plt.figure()
    plt.scatter(yte, preds, s=10)
    plt.xlabel("True")
    plt.ylabel("Predicted")
    plt.title("Linear Regression: True vs Predicted")
    lims = [min(yte.min(), preds.min()), max(yte.max(), preds.max())]
    plt.plot(lims, lims, 'r--')
    plt.tight_layout()
    plt.savefig("true_vs_pred.png")
    plt.close()

    # "Model card"
    card = {
        "model": "LinearRegression",
        "dataset": "California Housing (fallback to Diabetes if offline)",
        "task": "Tabular regression",
        "preprocessing": "StandardScaler on all numeric features",
        "target": "MedianHouseValue (or Diabetes target)",
        "metrics": {"RMSE": rmse, "MAE": mae},
        "intended_use": "Intro ML coursework; not for real-estate decisions",
        "limitations": [
            "Linear model; no interaction/nonlinearity modeling",
            "No feature engineering; sensitive to scale/outliers"
        ],
        "owner": "Student"
    }

    with open("model_card.json","w") as f:
        json.dump(card, f, indent=2)

    print("Saved: target_hist.png, true_vs_pred.png, model_card.json")

    return rmse, mae

# ---------------- Main ----------------
if __name__=="__main__":
    ds = load_dataset()
    quick_eda(ds.data, ds.target, ds.name)
    build_and_eval(ds.data, ds.target, ds.feature_names)


Dataset: California Housing | samples=20640 | features=8
Head:
    MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   

   Longitude  
0    -122.23  
1    -122.22  
2    -122.24  
Target stats: mean=2.069 std=1.154
RMSE: 0.7456 | MAE: 0.5332
Saved: target_hist.png, true_vs_pred.png, model_card.json
